<a href="https://colab.research.google.com/github/farsem12/Interactive-movie-recommender-for-new-users/blob/main/Initial_training_with_no_new_user.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# mounting required libraries
import numpy as np
import pandas as pd
from numpy import savetxt
import csv
import math
import random
import re
import sklearn
from google.colab import files
import progressbar as pb
import os

In [4]:
# connecting the google drive to google colab in order to load the MovieLens 100K dataset (previously uploaded on my drive)
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials


In [6]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


In [8]:
# Determining the type of dataset; The original one or the one with manually added users
# The original dataset has 6040 users
# Here we load the original dataset for training

raw_or_not = input ('Please type "0" if it is the your first time running this code and "1" if there is an updated version of MovieLens 100K dataset in your Drive:')
while raw_or_not != '0' and raw_or_not != '1':
  print('Error! please enter 0 or 1 as a no or yes, respectively.')
  raw_or_not = input('Do you already have an updated version of MovieLens 100K dataset in your Drive? if yes please enter 1 as yes and 0 as no: ')
print('Thanks for your clarification!')
if raw_or_not == '1':
  path = "/content/drive/MyDrive/ratings_uploaded_df.csv"
  ratings_downloaded_df = pd.read_csv(path, low_memory=False)
  print(ratings_downloaded_df.shape)
  downloaded_m = drive.CreateFile({'id':"14YYajuaair3k8lhuFjl0HhrsEqEKbsYl"})
  downloaded_u = drive.CreateFile({'id':"1jZXlSmma45Fnig_M8NBxl7uoCtlKCwaV"})
else:
  downloaded_r = drive.CreateFile({'id':"1jwG2grZIrWabRx0wUTNWkl53qNhP30y2"})
  downloaded_m = drive.CreateFile({'id':"14YYajuaair3k8lhuFjl0HhrsEqEKbsYl"})
  downloaded_u = drive.CreateFile({'id':"1jZXlSmma45Fnig_M8NBxl7uoCtlKCwaV"})

Please type "0" if it is the your first time running this code and "1" if there is an updated version of MovieLens 100K dataset in your Drive:0
Thanks for your clarification!


In [9]:
# loading the ratings, movies, and users datasets on colab
if raw_or_not == '0':
  downloaded_r.GetContentFile('ratings.dat')
downloaded_m.GetContentFile('movies.dat')
downloaded_u.GetContentFile('users.dat')

In [10]:
# reading the data
if raw_or_not == '1':
  ratings_list = ratings_downloaded_df
else:
  ratings_list = [i.strip().split("::") for i in open('ratings.dat', 'r').readlines()]
users_list = [i.strip().split("::") for i in open('users.dat', 'r').readlines()]
movies_list = [i.strip().split("::") for i in open('movies.dat', 'r').readlines()]

In [11]:
# converting lists to numpy arrays
ratings = np.array(ratings_list)
users = np.array(users_list)
movies = np.array(movies_list)

In [12]:
# converting numpy arrays to dataframes with four corresponding columns
ratings_df = pd.DataFrame(ratings, columns = ['user_id', 'movie_id', 'rating', 'timestamp'])
ratings_df = ratings_df._convert(numeric=True).fillna(1.0).astype(int)
movies_df = pd.DataFrame(movies_list, columns = ['movie_id', 'title', 'genres'])
movies_df = movies_df._convert(numeric=True).fillna(1.0)
movies_df['movie_id']= movies_df['movie_id'].astype(int)
users_df = pd.DataFrame(ratings, columns = ['user_id', 'sex', 'age', 'NI'])
users_df = ratings_df._convert(numeric=True).fillna(1.0).astype(int)



In [13]:
ratings_df.head(-1)

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
...,...,...,...,...
1000203,6040,1090,3,956715518
1000204,6040,1091,1,956716541
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746


In [14]:
movies_df.head(-1)

,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
3877,3947,Get Carter (1971),Thriller
3878,3948,Meet the Parents (2000),Comedy
3879,3949,Requiem for a Dream (2000),Drama
3880,3950,Tigerland (2000),Drama


In [15]:
# removing duplicate rows (based on first the same "user_id" and "movie_id")
# keeping the first one if the rating is the same and the most recent one if the ratings differ

ratings_df = ratings_df.drop_duplicates(subset=['user_id', 'movie_id', 'rating'], keep='first')
ratings_df = ratings_df.drop_duplicates(subset=['user_id', 'movie_id'], keep='last')



In [16]:
# crossing the users as rows, and ratings to each movie as columns to make a new and visually sound dataframe
R_df = ratings_df.pivot(index = 'user_id', columns ='movie_id', values = 'rating').fillna(0)
print(R_df.shape)
R_df.head(-1)

(6040, 3706)


movie_id,1,2,3,4,5,6,7,8,9,10,...,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
user_id,,,,,,,,,,,,,,,,,,,,,
1,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6035,4.0,0.0,1.0,2.0,1.0,0.0,3.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6036,0.0,0.0,0.0,2.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6037,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
# The introduction and calculation of a movie quality metric considering both the popularity (with a customary 80% weight) and rating (with a customary 20% weight) of a movie
counter = 0
k = 0.2
popular_movies= {}
dff = movies_df.merge(ratings_df).merge(users_df)
n_views = dff.groupby('movie_id', sort=False).movie_id.count()
ratings_c = dff.groupby('movie_id', sort=False).rating.mean()
weights = ((1-k)*(n_views/n_views.max()) + k*(ratings_c/ratings_c.max())).to_numpy()
print(weights[:10])
for column in R_df:
  counter +=1
  popular_movies[column] = weights[counter-1]






[0.65058798 0.29163958 0.23222138 0.14884975 0.18934845 0.37451883
 0.24330369 0.13645755 0.13007848 0.34885616]


In [18]:
# sorting movies based on this metric
import operator
popular_movies_sorted = sorted(popular_movies.items(),key=operator.itemgetter(1),reverse=True)

In [19]:
print(popular_movies_sorted[:10])

[(2858, 0.9726954492415403), (260, 0.8761641127193326), (1196, 0.8695020273724552), (1210, 0.8337278481550602), (2028, 0.7926306803112745), (589, 0.7805435396902379), (2571, 0.7770672769785956), (593, 0.7757065303488001), (480, 0.7741244873147521), (1198, 0.765806772621743)]


In [20]:
# calculating the proportiortion of each movie's weight to the sum of weights (to fascilitate further random selection of a movie)
weights_final = []
counter_2 = 0
for item in popular_movies_sorted:
  weights_final.append(item[1]/sum(weights))
print(weights_final[:10])

[0.001363169334485928, 0.0012278869520437963, 0.0012185504732356836, 0.0011684152905188506, 0.0011108202858516104, 0.0010938809453323718, 0.0010890091895008596, 0.0010871021917050042, 0.001084885061408432, 0.001073228325878977]


In [23]:
# creating a random movie proposal engine: Randomly proposing movies to the new-coming or an existing user, utilizing the previously calculated weights
proposal = []
proposal_for_random = []
for i in popular_movies_sorted:
  proposal_for_random.append(i[0])
weighted_random_mov_id = np.random.choice(proposal_for_random, size=50, replace=False, p = weights_final/sum(weights_final))
# the number of movies gathered in this phase is optional and is set to 50 here
for j in weighted_random_mov_id:
  tit = movies_df.loc[movies_df['movie_id'] == j, 'title'].item()
  genre= movies_df.loc[movies_df['movie_id'] == j, 'genres'].item()
  tige = (tit, genre, j)
  proposal.append(tige)
print(len(proposal))
print(proposal [:10])




50
[('Zero Effect (1998)', 'Comedy|Thriller', 1845), ('Full Metal Jacket (1987)', 'Action|Drama|War', 1222), ('L.A. Story (1991)', 'Comedy|Romance', 2108), ("Weekend at Bernie's (1989)", 'Comedy', 1091), ('To Die For (1995)', 'Comedy|Drama', 45), ('Heat (1995)', 'Action|Crime|Thriller', 6), ('Rocky II (1979)', 'Action|Drama', 2409), ('When Harry Met Sally... (1989)', 'Comedy|Romance', 1307), ('Teaching Mrs. Tingle (1999)', 'Comedy|Thriller', 2806), ('Dolores Claiborne (1994)', 'Drama|Thriller', 230)]


In [24]:
# All the following loops are designed in a way that any irrelevant or unsupported input would raise an error and a subsequent request for revision (until the provision of a correct input)
# This eliminates the risk of loosing any progress or termination of the running cell

In [25]:
# Generating a new user ID for the new user or retrieving the user ID of an existing one
new_or_not = input('Do you already have a user ID? if yes please enter 1 as yes and 0 as no   ')
while new_or_not != '0' and new_or_not != '1':
  print('Error! please enter 0 or 1 as a no or yes, respectively.')
  new_or_not = input('Do you already have a user ID? if yes please enter 1 as yes and 0 as no')
if new_or_not == '1':
  usid = input('please enter your user_id: ')
  usid_auth = re.fullmatch(r"^\d{1,7}$", usid)
  while usid_auth == None:
    usid = input('please enter your user_id correctly. it should be an integer between 1 and 10 million. Now enter your user_id: ')
    usid_auth = re.fullmatch(r"^\d{1,7}$", usid)
    while usid_auth != None:
      if int(usid) <= len(R_df):
        break
      else:
        usid = input('Your existing user ID has to be between 1 and %s' % len(R_df))
        usid_auth = re.fullmatch(r"^\d{1,7}$", usid)
print('thanks for your response! ')
if new_or_not == '0':
  new_user = input("please enter your nickname; You are gonna be called accordingly in further proceedings:")
  print("Hi %s!. Glad to see you here." %new_user)
  #usid_last = ratings_downloaded_df.iloc[-1,0]
  usid = ratings_df['user_id'].max() + 1
  print("your ID is : %s. remember yours for future updates" % str(usid))
else:
  new_user = usid

Do you already have a user ID? if yes please enter 1 as yes and 0 as no   1
please enter your user_id: 1
thanks for your response! 


In [26]:
# The engine tutorial
new_rating_or_not = input('Do you want to rate movies and complete your profile? enter 1 as a yes and 0 as a no: ')
if new_rating_or_not == '1' or '0':
  print('Thanks for your response!')
while new_rating_or_not != '0' and new_rating_or_not != '1':
  print('Error! please enter 0 or 1 as a no or yes, respectively.')
  new_rating_or_not = input('Do you want to rate movies and complete your profile? enter 1 as a yes and 0 as a no: ')
  if new_rating_or_not == '1' or new_rating_or_not == '0':
    print('Thanks for your response!')
if new_rating_or_not == '1':
  print("This movie recommender system requires your movie ratings to get into action. It is obligatory for you to rate at least 10 (The more the better provided that you have already wathced them!!) movies to solve the cold start problem")
  print("Instructions: First, you can rate any movie in the dataset as you wish. you can do so by searching for the whole or a part of your desired movie's name. Secend, if you wish, the system showes you a combination of the most popular movies in the movielens dataset (the most frequently rated; to increase the probability of you having watched them already) and some random movies.")
  print("%s!you got it? enter 1 as a yes an 0 as a no" %new_user)
  resp = input("your response:")
  while resp != '1':
    if resp == '0':
      print("lets try again!")
      print("Hey %s! you got it? enter 1 as a yes an 0 as a no" %new_user)
      resp = input("your response:")
    else:
      if resp != '1':
        print('Error! please enter 0 or 1 as a no or yes, respectively. ')
        resp = input("your response:")
  print ('Thanks for your understanding! ')


Do you want to rate movies and complete your profile? enter 1 as a yes and 0 as a no: 0
Thanks for your response!


In [27]:
# Provide the new user with a platform to rate his/her desired movie (if present in the movies dataset)
# Note that the dataset used here only contains movies up to 1999
count_movgen = 0
res_list = []
if new_rating_or_not == '1':
    while True:  # Use a while True loop for continuous interaction until the user decides to exit
        cntinue = input('Do you have any specific movie in mind for rating? Please type "1" as yes and "0" as no: ')
        if cntinue == '0':
            print('Thanks for your response!')
            break  # Exit the loop if the user doesn't want to rate any movies
        elif cntinue == '1':
            print('Thanks for your response!')
        else:
            print('Error! Please enter 0 or 1 as a no or yes, respectively.')
            continue  # Repeat the loop if the user provided an incorrect input

        while True:  # Use a nested while loop for continuous interaction until the user decides to go back to the main menu
            des_mov = input("Please search for the whole or a part of the movie's title you have in mind to rate (type 'exit' to quit, 'again' to go back to the main menu): ")
            if des_mov.lower() == 'exit':
                break  # Exit the nested loop and go back to the main menu
            elif des_mov.lower() == 'again':
                break  # Exit the nested loop and go back to the main menu

            movies_df['title'] = movies_df['title'].str.strip().str.lower().str.replace(' ', '_')
            movies_df['Popularity_rating_score'] = popular_movies

            if movies_df['title'].str.contains(des_mov, case=False).any():
                movies_df.loc[movies_df['title'].str.contains(des_mov), 'title'] = movies_df['title'].str.replace('_', ' ')
                print(movies_df.loc[movies_df['title'].str.contains(des_mov)].sort_values(by=['Popularity_rating_score'], ascending=False))

                while True:  # Use a nested while loop for continuous interaction until the user decides to go back to the main menu
                    movid = input('Please enter the movie ID of your desired movie (type "exit" to go back to the main menu): ')
                    if movid.lower() == 'exit' or movid.lower() == 'again' :
                        break  # Exit the nested loop and go back to the main menu
                    movid_auth = re.fullmatch(r"^\d{1,4}$", movid)
                    if movid_auth:
                        print('Success')
                    else:
                        print('Please enter a valid movie ID (integer between 1 and 9999).')
                        continue

                    while int(movid) > len(movies_df):
                        movid = input('The movie ID is out of range, please provide the correct one (type "exit" to go back to the main menu): ')
                        if movid.lower() == 'exit':
                            break  # Exit the nested loop and go back to the main menu
                        movid_auth = re.fullmatch(r"^\d{1,4}$", movid)
                        if movid_auth:
                            print('Success')
                        else:
                            print('Please enter a valid movie ID (integer between 1 and 9999).')

                    if movid.lower() == 'exit':
                        break  # Exit the nested loop and go back to the main menu

                    tit = movies_df.loc[movies_df['movie_id'] == int(movid), 'title'].item()
                    genre = movies_df.loc[movies_df['movie_id'] == int(movid), 'genres'].item()
                    res = input("Please rate %s from %s genre from 1 to 5 (type 'exit' to go back to the main menu, 'again' to rate another movie): " % (tit, genre))

                    if res.lower() == 'exit':
                        break  # Exit the nested loop and go back to the main menu
                    elif res.lower() == 'again':
                        break  # Exit the nested loop and go back to the main menu

                    while not res.isdigit() or int(res) not in range(1, 6):
                        print("ERROR! Please enter a valid integer from 1 to 5 for %s from %s genre." % (tit, genre))
                        res = input('Your response: ')
                        if res.lower() == 'exit':
                            break  # Exit the nested loop and go back to the main menu
                        elif res.lower() == 'again':
                            break  # Exit the nested loop and go back to the main menu

                    if res.lower() == 'exit':
                        break  # Exit the nested loop and go back to the main menu

                    count_movgen += 1
                    print('Congrats! %s response(s) recorded successfully!' % count_movgen)
                    res_list.append((int(movid), int(res)))
                    break  # Exit the nested loop and go back to the main menu
            else:
                print('The search query retrieved 0 results!')

        if des_mov.lower() == 'exit':
            break  # Exit the main loop and end the interactionno or yes, respectively.')
        if cntinue == '1' or cntinue == '0':
          print('Thanks for your response!')




In [28]:
print(res_list)

[]


In [29]:
movies_df['title'] = movies_df['title'].str.replace('_', ' ')

In [30]:
# # Inquiring the ratings of the user with successively random proposed movies (the more the quality index the higher the probability of being proposed sooner and thus the more chane
# the user can rate the movie)
if new_rating_or_not == '1':
  cntinue = input ('Do you want to rate our proposed films (based on the probability of a user having seen them)? please type "1" as a yes and "0" as a no: ')
  if cntinue == '1' or cntinue == '0':
    print('Thanks for your response!')
  while cntinue != '0' and cntinue != '1':
    cntinue = input('Error! please enter 0 or 1 as a no or yes, respectively.')
    if cntinue == '1' or cntinue == '0':
      print('Thanks for your response!')
  if cntinue == '1':
    print("please enter 's' if you have not seen the movie and proceed. Note that you can terminate the process of rating at any point by typing 'exit' or 'EXIT'.")
    for movgen in proposal:
      print("please rate %s from %s genre from 1 t0 5" % (movgen[0],movgen[1]))
      res = input('your response:')
      while((res.isdigit() == True and int(res) in range(1,6)) == False):
        if res.lower() == 'exit':
          break
        elif res.lower() == 's':
          break
        print("ERROR! please enter a valid integer from 1 to 5 for %s from %s genre " % (movgen[0],movgen[1]))
        res = input('Your response:')
      if res.lower() == 'exit':
        break
      if res.lower() != 's':
        count_movgen +=1
        res_list.append((movgen[2], int(res)))
        print('Congrats! %s response(s) recorded successfully!' % count_movgen)




In [31]:
if new_rating_or_not == '1':
  print(res_list)

In [32]:
# standardizing the format of the user responses
if new_rating_or_not == '1':
  X_new_user = []
  for res in res_list:
    X_new_user.append([usid, res[0], res[1], 1000])



In [33]:
# adding the user responses to the existing user-movie ratings list
if new_rating_or_not == '1':
  ratings_final = np.vstack([ratings, X_new_user])
  print(ratings_final)

In [34]:
# uploading the updated dataset in the google drive for future uses
if new_rating_or_not == '1':
  ratings_uploaded = ratings_final
  ratings_uploaded_df = pd.DataFrame(ratings_uploaded, columns = ratings_df.columns)
  ratings_uploaded_df.to_csv('/content/drive/My Drive/ratings_uploaded_df.csv', index=False)

In [35]:
# converting numpy arrays to numpy databases again!!
if new_rating_or_not == '1':
  ratings_df_new = pd.DataFrame(ratings_final, columns = ['user_id', 'movie_id', 'rating', 'timestamp'])
  ratings_df_new = ratings_df_new._convert(numeric=True).fillna(1.0).astype(int)
  print(ratings_df_new)
else:
  ratings_df_new = ratings_df

In [36]:
# removing duplicate rows, if any is present (based on first the same "user_id" and "movie_id")
# keeping the first one if the rating is the same and the most recent one if the ratings differ

ratings_df_new = ratings_df_new.drop_duplicates(subset=['user_id', 'movie_id', 'rating'], keep='first')
ratings_df_new = ratings_df_new.drop_duplicates(subset=['user_id', 'movie_id'], keep='last')

In [37]:
ratings_df_new.head(-1)

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
...,...,...,...,...
1000203,6040,1090,3,956715518
1000204,6040,1091,1,956716541
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746


In [38]:
# crossing the users as rows and ratings to each movie as columns to make a new and visually sound dataframe again!!
R_df_new = ratings_df_new.pivot(index = 'user_id', columns ='movie_id', values = 'rating').fillna(0)
print(R_df_new.shape)
R_df_new.head(-1)

(6040, 3706)


movie_id,1,2,3,4,5,6,7,8,9,10,...,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
user_id,,,,,,,,,,,,,,,,,,,,,
1,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6035,4.0,0.0,1.0,2.0,1.0,0.0,3.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6036,0.0,0.0,0.0,2.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6037,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [39]:
#using TF-IDF for the content-based movie recommendation section
# The next seven cells are from this reference: https://github.com/AlexanderNixon/Machine-learning-reads/blob/master/Movie-content-based-recommender-using-tf-idf.ipynb

In [40]:
import matplotlib.pyplot as plt
from itertools import combinations
import seaborn as sns
print(movies_df.genres.str.split('|').explode())
genre_popularity = (movies_df.genres.str.split('|')
                      .explode()
                      .value_counts()
                      .sort_values(ascending=False))
genre_popularity.head(10)



0        Animation
0       Children's
0           Comedy
1        Adventure
1       Children's
           ...    
3879         Drama
3880         Drama
3881         Drama
3882         Drama
3882      Thriller
Name: genres, Length: 6408, dtype: object


Drama         1603
Comedy        1200
Action         503
Thriller       492
Romance        471
Horror         343
Adventure      283
Sci-Fi         276
Children's     251
Crime          211
Name: genres, dtype: int64

In [41]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [42]:
tf = TfidfVectorizer(analyzer=lambda s: (c for i in range(1,4)
                                             for c in combinations(s.split('|'), r=i)))
tfidf_matrix = tf.fit_transform(movies_df['genres'])
tfidf_matrix.shape

(3883, 353)

In [43]:
# showing a 10 * 10 sample
pd.DataFrame(tfidf_matrix.todense(), columns=tf.get_feature_names_out(), index=movies_df.title).sample(10, axis=1).sample(10, axis=0)

,"(Adventure, Mystery, Sci-Fi)","(Crime, Film-Noir)","(Drama, Sci-Fi, War)","(Action, Comedy, Fantasy)","(Romance,)","(Action, Sci-Fi, Thriller)","(Drama, Mystery, Romance)","(Adventure, Sci-Fi)","(Adventure, Comedy, Sci-Fi)","(Mystery, Romance)"
title,,,,,,,,,,
Body Parts (1991),0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
Naked in New York (1994),0.0,0.0,0.0,0.0,0.568145,0.0,0.0,0.0,0.0,0.0
Mr. Mom (1983),0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
Tall Tale (1994),0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
"Trigger Effect, The (1996)",0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
Trouble in Paradise (1932),0.0,0.0,0.0,0.0,0.568145,0.0,0.0,0.0,0.0,0.0
Coogan's Bluff (1968),0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
"Hard Day's Night, A (1964)",0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
Among Giants (1998),0.0,0.0,0.0,0.0,0.579633,0.0,0.0,0.0,0.0,0.0


In [44]:
# using cosine similarity to identify the extent of similarity between all possible combinations of any two movies
from sklearn.metrics.pairwise import cosine_similarity
cosine_sim = cosine_similarity(tfidf_matrix)

In [45]:
cosine_sim_df = pd.DataFrame(cosine_sim, index=movies_df['title'], columns=movies_df['title'])
print('Shape:', cosine_sim_df.shape)
cosine_sim_df.sample(5, axis=1).round(2)

Shape: (3883, 3883)


title,My Favorite Season (1993),American Movie (1999),Broken English (1996),In the Company of Men (1997),"Gay Divorcee, The (1934)"
title,,,,,
Toy Story (1995),0.00,0.0,0.00,0.00,0.03
Jumanji (1995),0.00,0.0,0.00,0.00,0.00
Grumpier Old Men (1995),0.00,0.0,0.00,0.00,0.42
Waiting to Exhale (1995),0.39,0.0,0.39,0.39,0.08
Father of the Bride Part II (1995),0.00,0.0,0.00,0.00,0.17
...,...,...,...,...,...
Meet the Parents (2000),0.00,0.0,0.00,0.00,0.17
Requiem for a Dream (2000),1.00,0.0,1.00,1.00,0.00
Tigerland (2000),1.00,0.0,1.00,1.00,0.00


In [46]:
cosine_sim_itemID_df = pd.DataFrame(cosine_sim, index=movies_df['movie_id'], columns=movies_df['movie_id'])
print('Shape:', cosine_sim_df.shape)
cosine_sim_df.sample(5, axis=1).round(2)

Shape: (3883, 3883)


title,See the Sea (Regarde la mer) (1997),Groundhog Day (1993),"Bye Bye, Love (1995)",Dying Young (1991),Sleeping Beauty (1959)
title,,,,,
Toy Story (1995),0.00,0.07,0.17,0.00,0.34
Jumanji (1995),0.00,0.00,0.00,0.00,0.08
Grumpier Old Men (1995),0.00,1.00,0.40,0.33,0.00
Waiting to Exhale (1995),0.39,0.18,0.45,0.14,0.00
Father of the Bride Part II (1995),0.00,0.40,1.00,0.00,0.00
...,...,...,...,...,...
Meet the Parents (2000),0.00,0.40,1.00,0.00,0.00
Requiem for a Dream (2000),1.00,0.00,0.00,0.35,0.00
Tigerland (2000),1.00,0.00,0.00,0.35,0.00


In [47]:
R_final = np.array(R_df_new)
print(R_final.shape)

(6040, 3706)


In [48]:
# implementing matrix factorization as the preferred collaborative movie recommendation approach
# the next few cells are inspired from this reference: https://towardsdatascience.com/recommendation-system-matrix-factorization-d61978660b4b

In [49]:
# Defining a regular matrix factorization (to update the user-K (P) and K-movie (Q) matrices from scratch)
def matrix_factorization(R, P, Q, K, steps=20, alpha=0.0005, beta=0.02):
    Q = Q.T
    counter = 0
    for step in pb.progressbar(range(steps)):
        for i in range(len(R)):
            for j in range(len(R[i])):
                if R[i][j] > 0:
                    # calculate error
                    eij = R[i][j] - np.dot(P[i,:],Q[:,j])

                    for k in range(K):
                        # calculate gradient with a and beta parameter
                        P[i][k] = P[i][k] + alpha * (2 * eij * Q[k][j] - beta * P[i][k])
                        Q[k][j] = Q[k][j] + alpha * (2 * eij * P[i][k] - beta * Q[k][j])

        eR = np.dot(P,Q)

        e = 0

        for i in range(len(R)):

            for j in range(len(R[i])):

                if R[i][j] > 0:

                    counter += 1

                    e = e + pow(R[i][j] - np.dot(P[i,:],Q[:,j]), 2)

        # 0.001: local minimum
        if e < 0.001:

            break

    return P, Q.T, e , counter

In [50]:
# Defining the modified MF for the new_user (or the few new users, as long as the previously calculated weights outweigh the new changes)
# This is achieved via utilizing a new function that only optimizes the corresponding randomly initialized row of the P matrix for the new user
# We assume that adding a new user or updating the profile of an old user does not alter the Q matrix (movie - latent factors matrix) significantly
# Thus, for rapid updating of the model, we use the previously trained weights for all other users and then add another row with randomly initialized values for the corresponding row of the P matrix for the user
# Finally, the model gets updated rapidly and conveniently

def matrix_factorization_modified(R, P, Q, K, new_user_id, steps=30, alpha=0.0005, beta=0.02):
    Q = Q.T
    counter_mod = 0
    for step in pb.progressbar(range(steps)):
      i = new_user_id - 1
      for j in range(1, len(R[i])):
        if R[i][j] > 0:
          eij = R[i][j] - np.dot(P[i,:],Q[:,j])
          for k in range(K):
            P[i][k] = P[i][k] + alpha * (2 * eij * Q[k][j] - beta * P[i][k])
      eR = np.dot(P,Q)
      e = 0
      for j in range(len(R[i])):
        if R[i][j] > 0:
          counter_mod += 1
          e = e + pow(R[i][j] - np.dot(P[i,:],Q[:,j]), 2)

      if e < 0.001:
        break
    return P, Q.T, e , counter_mod

In [51]:
# checking if we have previously saved weights from our former iterations in our drive
first_train_or_not = input('please write "1" if you want to train the model for the first time or write "0" if you need the previously trained weights for convenience:')
while first_train_or_not != '0' and first_train_or_not != '1':
  print('Error! please enter 0 or 1 as a no or yes, respectively.')
  first_train_or_not = input('Is it the first training of the model? if yes please enter 1 as yes and 0 as no')
print ('TNX!!')

please write "1" if you want to train the model for the first time or write "0" if you need the previously trained weights for convenience:1
TNX!!


In [52]:
# opening the previously trained weights of the model (saved in the Google Drive)
if first_train_or_not == "0":
      with open('/content/drive/My Drive/nP.csv', 'r') as f:
        nP_retrieved = np.loadtxt(f)
      with open('/content/drive/My Drive/nQ.csv', 'r') as f:
        nQ_retrieved = np.loadtxt(f)
      with open('/content/drive/My Drive/parameters.csv', 'r') as f:
        parameters = np.loadtxt(f)
        E = parameters[0]
        tot_num_votes = parameters[1]
      nR_retrieved  = np.dot(nP_retrieved, nQ_retrieved.T)
      print(nR_retrieved.shape)
      print(E)
      print(tot_num_votes)

In [53]:
# Initilaizing or loading the previous weights accordingly

if first_train_or_not == "1":
  K = 4
  n = len(R_df_new.axes[0])
  m = len(R_df_new.axes[1])
  P = np.random.rand(n,K)
  Q = np.random.rand(m,K)

else:
  if new_or_not == "0":
    if len(res_list) != 0:
      nP_new_row = np.random.rand(1,4)
      nP_retrieved_new = np.vstack([nP_retrieved, nP_new_row])
      P = nP_retrieved_new

  else:
    P = nP_retrieved
  Q = nQ_retrieved
  K = 4

In [54]:
print(P)

[[0.89648409 0.2358961  0.10686879 0.25652541]
 [0.03297565 0.69478283 0.48869078 0.76947011]
 [0.16793008 0.94218848 0.13008255 0.09654332]
 ...
 [0.75030856 0.16559075 0.24965258 0.90871154]
 [0.33306761 0.19346516 0.58556493 0.44255528]
 [0.55363966 0.53578033 0.80075279 0.72793953]]


In [55]:
# conducting the matrix factorization

if new_or_not == "0": # specifying the status of the user: whether the user is a new-comer!

  if first_train_or_not == "0": # determine whether the model has been pre-trained!

    if len(res_list) != 0: # determine whether the new user has rated any movie!

        nP, nQ, E_mod, tot_num_votes_mod = matrix_factorization_modified (R_final, P, Q, K, usid, steps=1000, alpha= 0.0005, beta=0.02)

        rmse = np.sqrt((E + E_mod) / (tot_num_votes_mod + tot_num_votes)) # calculating RMSE (root mean squared error) to assess the model accuracy

        tot_num_votes += tot_num_votes_mod # updating the total number of votes of all users

        E += E_mod   # updating the total error


    else:

        pass


  else:

    nP, nQ, E, tot_num_votes = matrix_factorization(R_final, P, Q, K, steps= 20, alpha= 0.0002, beta=0.01)  # Fast learning and updating the predictions for the new user with the pre-trained weights

    rmse = np.sqrt(E / tot_num_votes)


elif new_or_not == "1":

  if first_train_or_not == "1":

    nP, nQ, E, tot_num_votes = matrix_factorization(R_final, P, Q, K, steps= 20, alpha= 0.0002, beta=0.02)

    rmse = np.sqrt(E / tot_num_votes)


  else:

    if len(res_list) != 0:

      nP, nQ, E_mod, tot_num_votes_mod = matrix_factorization_modified (R_final, P, Q, K, int(usid), steps=100, alpha= 0.0005, beta=0.02)

      rmse = np.sqrt((E + E_mod) / (len(res_list) + tot_num_votes))

      tot_num_votes += len(res_list)

      E += E_mod  # This is an approximation

100% (20 of 20) |########################| Elapsed Time: 0:19:50 Time:  0:19:50


In [56]:
print("nP shape is : " + str(nP.shape))
print("nQ shape is : " + str(nQ.shape))
print(" RMSE is: %s  " % rmse)
nr = np.dot(nP,nQ.T)

nP shape is : (6040, 4)
nQ shape is : (3706, 4)
 RMSE is: 0.20719621648883313  


In [ ]:
# As shown above,the RMSE is about 0.2 after only 20 iteratons (which is not bad considering the range of our values)
# This took about 20 minutes!!!

In [63]:
# Uploading the weights for further use in Drive (overwriting the previous files)

with open('/content/drive/My Drive/nP.csv', 'w') as f:
    np.savetxt(f, nP)
with open('/content/drive/My Drive/nQ.csv', 'w') as f:
    np.savetxt(f, nQ)
with open('/content/drive/My Drive/parameters.csv', 'w') as f:
    np.savetxt(f, np.array([E, tot_num_votes]))


In [57]:
nr = np.dot(nP,nQ.T)
print(nr)

[[4.26226156 3.42925606 3.23764778 ... 3.03515175 3.13985101 4.06399563]
 [4.01009803 2.90344902 2.94176534 ... 2.66634347 2.88774801 3.59694919]
 [3.91543146 2.85503501 2.92114948 ... 2.82881567 2.81311128 3.46195775]
 ...
 [3.52432817 2.7860529  2.69735287 ... 2.32066437 2.59210103 3.42986206]
 [4.04893824 3.14494493 2.91310105 ... 2.73164689 2.96687554 3.71709007]
 [3.88449364 2.98483009 2.82052894 ... 2.6375445  2.83712706 3.55667309]]


In [58]:
preds_df = pd.DataFrame(nr, columns = R_df_new.columns)
print(preds_df.shape)

(6040, 3706)


In [59]:
def recommend_movies(predictions_df, user_id, movies_df, original_ratings_df, num_recommendations=5):
    # inspired by: https://beckernick.github.io/matrix-factorization-recommender/
    # Get and sort the user's predictions
    user_row_number = user_id - 1 # user_id starts at 1, not 0
    sorted_user_predictions = preds_df.iloc[user_row_number].sort_values(ascending=False) # user_id starts at 1

    # Get the user's data and merge in the movie information.
    user_data = original_ratings_df[original_ratings_df.user_id == (user_id)]
    user_full = (user_data.merge(movies_df, how = 'left', left_on = 'movie_id', right_on = 'movie_id').
                     sort_values(['rating'], ascending=False)
                 )

    print ('User %s has already rated %s movies.'%(user_id, user_full.shape[0]))
    print ('Recommending highest %s predicted ratings movies not already rated.' % num_recommendations)

    # Recommend the highest predicted rating movies that the user hasn't seen yet.
    recommendations = (movies_df[~movies_df['movie_id'].isin(user_full['movie_id'])].
         merge(pd.DataFrame(sorted_user_predictions).reset_index(), how = 'left',
               left_on = 'movie_id',
               right_on = 'movie_id').
         rename(columns = {user_row_number: 'Predictions'}).
         sort_values('Predictions', ascending = False).
                       iloc[:num_recommendations, :-1]
                      )

    return user_full, recommendations

In [61]:
# The number of predictions is can be set as desired. Here it has been set to 10.
already_rated, predictions = recommend_movies(preds_df, 200, movies_df, ratings_df_new, 10)

User 200 has already rated 26 movies.
Recommending highest 10 predicted ratings movies not already rated.


In [62]:
already_rated


,user_id,movie_id,rating,timestamp,title,genres
13,200,170,5,976957420,Hackers (1995),Action|Crime|Thriller
8,200,3408,5,976958202,Erin Brockovich (2000),Drama
23,200,1059,5,976957604,William Shakespeare's Romeo and Juliet (1996),Drama|Romance
18,200,2694,5,976957992,Big Daddy (1999),Comedy
12,200,2572,5,976957420,10 Things I Hate About You (1999),Comedy|Romance
25,200,2041,5,976958135,Condorman (1981),Action|Adventure|Children's|Comedy
3,200,3051,5,976957970,Anywhere But Here (1999),Drama
0,200,2997,4,976957992,Being John Malkovich (1999),Comedy
5,200,940,4,976957366,"Adventures of Robin Hood, The (1938)",Action|Adventure
24,200,3751,4,976958135,Chicken Run (2000),Animation|Children's|Comedy


In [63]:
predictions # predictions for the new user 200

,movie_id,title,genres
709,720,Wallace & Gromit: The Best of Aardman Animatio...,Animation
908,922,Sunset Blvd. (a.k.a. Sunset Boulevard) (1950),Film-Noir
845,858,"Godfather, The (1972)",Action|Crime|Drama
2930,3022,"General, The (1927)",Comedy
738,750,Dr. Strangelove or: How I Learned to Stop Worr...,Sci-Fi|War
1127,1148,"Wrong Trousers, The (1993)",Animation|Comedy
1184,1207,To Kill a Mockingbird (1962),Drama
3341,3435,Double Indemnity (1944),Crime|Film-Noir
1157,1178,Paths of Glory (1957),Drama|War
927,942,Laura (1944),Crime|Film-Noir|Mystery


In [64]:
# using TF-IDF to consider genres as a priority metric to reconsider the suggestions
# It is achieved by calculating the sum of all the simple multiplications of each recommended movie's cosine similarity with all the previously rated movies, by the user rating
def content_based_fil(predictions, already_rated_by_user, cosine_df_of_movies):
  sigmacosine = []
  for movie in predictions['movie_id'].to_numpy():
    sigma = 0
    for item in already_rated_by_user['movie_id'].to_numpy():
      sigma = sigma + cosine_df_of_movies.loc[movie, item] * float(already_rated_by_user.loc[already_rated_by_user['movie_id'] == item, 'rating'])
    sigmacosine.append(sigma)

  predictions_new = predictions.assign(metric = sigmacosine).sort_values(by = ['metric'], ascending = False)
  return predictions_new


In [65]:
modified_pred = content_based_fil(predictions, already_rated, cosine_sim_itemID_df)

In [66]:
# Generating the final recommendations
modified_pred

,movie_id,title,genres,metric
2930,3022,"General, The (1927)",Comedy,32.736696
1127,1148,"Wrong Trousers, The (1993)",Animation|Comedy,13.045978
1184,1207,To Kill a Mockingbird (1962),Drama,11.757346
845,858,"Godfather, The (1972)",Action|Crime|Drama,5.000841
1157,1178,Paths of Glory (1957),Drama|War,3.259143
709,720,Wallace & Gromit: The Best of Aardman Animatio...,Animation,2.535119
738,750,Dr. Strangelove or: How I Learned to Stop Worr...,Sci-Fi|War,1.276145
3341,3435,Double Indemnity (1944),Crime|Film-Noir,1.115926
927,942,Laura (1944),Crime|Film-Noir|Mystery,0.641124
908,922,Sunset Blvd. (a.k.a. Sunset Boulevard) (1950),Film-Noir,0.000000


In [2]:
# references:
# 1 - https://github.com/AlexanderNixon/Machine-learning-reads/blob/master/Movie-content-based-recommender-using-tf-idf.ipynb
# 2 - https://towardsdatascience.com/recommendation-system-matrix-factorization-d61978660b4b
# 3 - https://beckernick.github.io/matrix-factorization-recommender/